<a href="https://colab.research.google.com/github/isegura/iso4simplify/blob/main/metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluación de simplificación de textos (Prompting + LLM)

Este notebook (Google Colab) carga `test.csv` y `pred.csv` (o un único CSV combinado), alinea por `pair_id` y evalúa:

**Métricas de simplificación:** BLEU, SARI, BERTScore, MeaningBERT y LENS  
**Legibilidad:** (según idioma) FKGL, Flesch Reading Ease y SMOG (o adaptaciones ES/FR cuando aplique)

Al final, guarda un CSV con las puntuaciones:
`scores_{dataset}_{model}_{tipo_prompt}_{strategy_prompt}.csv`


## 1) Configuración


In [23]:
#@title Parámetros (edita si lo necesitas)
DATASET = "tsar2024"          #@param ["tsar2024","clara-med","cochrane","meds","wiki"]
MODEL_NAME = "gemma-3-12b-it" #@param {type:"string"}
PROMPT_STRATEGY = "one"      #@param ["zero","one","few"]
PROMPT_TYPE = "ISO"           #@param ["ISO","2steps","brief"]


# Rutas (Drive) o nombres (si subes con el uploader)
TEST_CSV_PATH = ""      #@param {type:"string"}
PRED_CSV_PATH = ""      #@param {type:"string"}
COMBINED_CSV_PATH = ""  #@param {type:"string"}

# Columnas: por defecto asumimos (pero el notebook intenta autodetectar).
# Si tu CSV usa otros nombres, edítalos aquí.
COL_PAIR_ID = "pair_id"         #@param {type:"string"}
COL_COMPLEX = "complex"         #@param {type:"string"}
COL_REFERENCE = "simple"       #@param {type:"string"}
COL_PREDICTION = "prediction"   #@param {type:"string"}


lang='en'
DATASET2LANG_DEFAULT = {
    "tsar2024": "en",
    "clara-med": "es",
    "cochrane": "fr",
    "meds": "fr",
    "wiki": "fr",
}
if DATASET in DATASET2LANG_DEFAULT.keys():
    lang = DATASET2LANG_DEFAULT[DATASET]
print(DATASET,lang)


tsar2024 en


## 2) Instalación de dependencias


In [6]:
!pip -q install sacrebleu bert-score textstat transformers torch
!pip install -q -U sentence-transformers


## 3) Carga de ficheros (subida manual) y del dataset


In [9]:
from google.colab import files
import os

print(
    f"Carga el fichero con las predicciones.\n"
    f"Debe incluir las columnas: {COL_PREDICTION}, {COL_COMPLEX} y {COL_REFERENCE}"
)

def upload_file():
    uploaded = files.upload()
    return list(uploaded.keys())

uploaded_files = upload_file()

# Validamos que solo se suba un fichero
if len(uploaded_files) != 1:
    raise ValueError("Debes subir exactamente un único fichero.")

# Construimos ruta absoluta en Colab
file_path = os.path.join("/content", uploaded_files[0])

print("✅ file_path =", file_path)

Carga el fichero con las predicciones.
Debe incluir las columnas: prediction, complex y simple


Saving predictions_gemma-3-12b-it_docs_one.csv to predictions_gemma-3-12b-it_docs_one.csv
✅ file_path = /content/predictions_gemma-3-12b-it_docs_one.csv


In [10]:
import pandas as pd

# Cargar ficheros
df = pd.read_csv(file_path)
df = df[[COL_PAIR_ID, COL_COMPLEX, COL_REFERENCE, COL_PREDICTION]].copy()


print("✅ Dataset cargado. Tamaño:", df.shape)
print(df.head(3))
# Extraer columnas
sources = df["complex"].astype(str).tolist()
references = df["simple"].astype(str).tolist()
predictions = df["prediction"].astype(str).tolist()

✅ Dataset cargado. Tamaño: (117, 4)
    pair_id                                            complex  \
0  CD001096  Twenty-eight studies (reporting a total of thi...   
1  CD006466  Of 12,620 identified citations, 10 RCTs fulfil...   
2  CD003416  We included 17 RCTs in the review (2505 women)...   

                                              simple  \
0  The reminders improved physician practices by ...   
1  The studies used two types of blood thinner:\n...   
2  We found 17 randomised controlled trials with ...   

                                          prediction  
0  Text: We studied 27 research reports involving...  
1  We looked at many studies about blood thinners...  
2  Text: We examined 17 studies (2505 women). The...  


## 4) Métricas de simplificación (BLEU, SARI, BERTScore, MeaningBERT, LENS)


In [29]:
scores={}

### BLEU Y SARI
*texto en cursiva*

In [33]:
import pandas as pd
import sacrebleu
import numpy as np



# ======================
# 🔵 BLEU
# ======================
# BLEU
bleu = sacrebleu.corpus_bleu(predictions, [references]).score
print(f"BLEU: {bleu:.2f}")
scores["BLEU"]=round(bleu,2)

# ======================
# 🟢 SARI  (implementación sin easse)
# ======================
def sari_score(orig_sents, sys_sents, refs_sents):
    def get_ngrams(sentence, n):
        tokens = sentence.split()
        return [" ".join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

    def f1(tp, denom_p, denom_r):
        p = tp / denom_p if denom_p else 0.0
        r = tp / denom_r if denom_r else 0.0
        return (2*p*r/(p+r)) if (p+r) else 0.0

    total = 0.0
    for orig, sys, ref in zip(orig_sents, sys_sents, refs_sents):
        add, keep, dele = [], [], []
        for n in range(1, 5):
            o = set(get_ngrams(orig, n))
            s = set(get_ngrams(sys, n))
            r = set(get_ngrams(ref, n))

            # ADD
            s_add = s - o
            r_add = r - o
            tp = len(s_add & r_add)
            add.append(f1(tp, len(s_add), len(r_add)))

            # KEEP
            s_keep = s & o
            r_keep = r & o
            tp = len(s_keep & r_keep)
            keep.append(f1(tp, len(s_keep), len(r_keep)))

            # DELETE
            s_del = o - s
            r_del = o - r
            tp = len(s_del & r_del)
            dele.append(f1(tp, len(s_del), len(r_del)))

        total += (np.mean(add) + np.mean(keep) + np.mean(dele)) / 3.0

    return (total / len(orig_sents)) * 100.0 if orig_sents else 0.0


sari = sari_score(sources, predictions, references)
print(f"SARI: {sari:.2f}")
scores["SARI"]=round(sari,2)


BLEU: 3.88
SARI: 32.50


### BERT_SCORE


In [34]:
from bert_score import score

if lang == "en":
    model_type = "roberta-large"
elif lang == "fr":
    model_type = "camembert-large"
else:
    model_type = "xlm-roberta-large"  # multilingüe por defecto

print("🤖 Modelo seleccionado:", model_type)

P, R, F1 = score(
    predictions,
    references,
    model_type=model_type,
    lang=lang,  # cambia si no es español
    verbose=True
)

print("BERTScore Precision:", round(P.mean().item(),4))
print("BERTScore Recall:", round(R.mean().item(),4))
print("BERTScore F1:", round(F1.mean().item(),4))
scores["BERTScore Precision:"]=round(P.mean().item(),4)
scores["BERTScore Recall:"]=round(R.mean().item(),4)
scores["BERTScore F1:"]=round(F1.mean().item(),4)


🤖 Modelo seleccionado: roberta-large


Loading weights:   0%|          | 0/389 [00:00<?, ?it/s]

RobertaModel LOAD REPORT from: roberta-large
Key                             | Status     | 
--------------------------------+------------+-
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
pooler.dense.bias               | MISSING    | 
pooler.dense.weight             | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 9.04 seconds, 12.94 sentences/sec
BERTScore Precision: 0.8654037714004517
BERTScore Recall: 0.8296453952789307
BERTScore F1: 0.8466267585754395


### MeaningBERT


In [36]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

if lang == "en":
    model_name = "davebulaval/meaningbert"
else:
    # modelo multilingüe para preservación semántica
    model_name = "joeddav/xlm-roberta-large-xnli"

print("Modelo MeaningBERT seleccionado:", model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
model.eval()
def _get_entailment_index(model):
    """
    Para modelos NLI: encuentra el índice de 'entailment' de forma robusta usando id2label.
    Si no lo encuentra, asume el orden típico: [contradiction, neutral, entailment] => entailment = 2
    """
    id2label = getattr(model.config, "id2label", None) or {}
    for idx, lab in id2label.items():
        if isinstance(lab, str) and "entail" in lab.lower():
            return int(idx)
    return 2

def compute_meaning_metric(refs, preds, batch_size=16, max_length=256):
    scores = []
    entail_idx = _get_entailment_index(model)

    for i in range(0, len(refs), batch_size):
        batch_refs = refs[i:i+batch_size]
        batch_preds = preds[i:i+batch_size]

        inputs = tokenizer(
            batch_refs,
            batch_preds,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length,
        ).to(device)

        with torch.no_grad():
            logits = model(**inputs).logits

            # Caso MeaningBERT (a veces 1 logit) -> sigmoid
            if logits.shape[-1] == 1:
                batch_scores = torch.sigmoid(logits).squeeze(-1)

            # Caso clasificación binaria (2 logits) -> prob clase positiva (idx=1)
            elif logits.shape[-1] == 2:
                probs = torch.softmax(logits, dim=-1)
                batch_scores = probs[:, 1]

            # Caso NLI (3 logits típico) -> prob entailment
            else:
                probs = torch.softmax(logits, dim=-1)
                batch_scores = probs[:, entail_idx]

        scores.extend(batch_scores.detach().cpu().tolist())

    return float(sum(scores) / len(scores)) if scores else 0.0

meaning_complex_simple = compute_meaning_metric(sources, references, batch_size=16)
print("MeaningBERT (complex → simple):", round(meaning_complex_simple, 4))
scores["MeaningBERT complex-simple"]=round(meaning_complex_simple, 4)

meaning_complex_pred = compute_meaning_metric(sources, predictions, batch_size=16)
print("MeaningBERT (complex → prediction):", round(meaning_complex_pred, 4))
scores["MeaningBERT complex-prediction"]=round(meaning_complex_pred, 4)

meaning_ref_pred = compute_meaning_metric(references, predictions, batch_size=16)
print("MeaningBERT (simple → prediction):", round(meaning_ref_pred, 4))
scores["MeaningBERT simple-prediction"]=round(meaning_ref_pred, 4)


Modelo MeaningBERT seleccionado: davebulaval/meaningbert


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

MeaningBERT (complex → simple): 0.9977
MeaningBERT (complex → prediction): 0.9614
MeaningBERT (simple → prediction): 0.9537


### Similitud coseno con sentence embeddings


In [37]:
from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')


complex_emb = embedder.encode(sources, convert_to_numpy=True, batch_size=32)
ref_emb = embedder.encode(references, convert_to_numpy=True, batch_size=32)
pred_emb = embedder.encode(predictions, convert_to_numpy=True, batch_size=32)


cos_complex_ref = cosine_similarity(complex_emb, ref_emb).diagonal()
semsim_complex_ref = round(np.mean(cos_complex_ref),4)
print("Semantic Similarity (complex → simple):", semsim_complex_ref)
scores["Semantic Similarity complex-simple"]=semsim_complex_ref



cos_complex_pred = cosine_similarity(complex_emb, pred_emb).diagonal()
semsim_complex_pred = round(np.mean(cos_complex_pred),4)

print("Semantic Similarity (complex → prediction):",
      np.mean(cos_complex_pred))
scores["Semantic Similarity complex-pred"]=semsim_complex_ref


cos_ref_pred = cosine_similarity(ref_emb, pred_emb).diagonal()
semsim_ref_pred = round(np.mean(cos_ref_pred),4)

print("Semantic Similarity (reference → prediction):", semsim_ref_pred)
scores["Semantic Similarity reference-pred"]=semsim_ref_pred


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Semantic Similarity (complex → simple): 0.7774
Semantic Similarity (complex → prediction): 0.5910318
Semantic Similarity (reference → prediction): 0.5627


## 6) Métricas de legibilidad (según idioma)


In [38]:
import numpy as np
import textstat

if lang not in {"en","es","fr"}:
    print(f"⚠️ Idioma '{lang}' no soportado en este notebook. Usaré 'en'.")
    lang = "en"

try:
    textstat.set_lang(lang)
except Exception:
    pass

def kandel_moles_french(text: str) -> float:
    """Kandel & Moles (adaptación francesa de Flesch): 207 - 1.015*ASL - 73.6*ASW"""
    sentences = max(1, textstat.sentence_count(text))
    words = max(1, textstat.lexicon_count(text, removepunct=True))
    syllables = max(1, textstat.syllable_count(text))
    asl = words / sentences
    asw = syllables / words
    return 207.0 - (1.015 * asl) - (73.6 * asw)

def avg_metric(texts, fn):
    vals = []
    for t in texts:
        try:
            vals.append(float(fn(t)))
        except Exception:
            vals.append(np.nan)
    return float(np.nanmean(vals))

pred_texts = predictions

if lang == "en":
    scores["FKGL"] = avg_metric(pred_texts, textstat.flesch_kincaid_grade)
    scores["FleschReadingEase"] = avg_metric(pred_texts, textstat.flesch_reading_ease)
    scores["SMOG"] = avg_metric(pred_texts, textstat.smog_index)

elif lang == "es":
    scores["FernandezHuerta"] = avg_metric(pred_texts, getattr(textstat, "fernandez_huerta", lambda x: np.nan))
    scores["SzigrisztPazos"]  = avg_metric(pred_texts, getattr(textstat, "szigriszt_pazos", lambda x: np.nan))
    scores["GutierrezPolini"] = avg_metric(pred_texts, getattr(textstat, "gutierrez_polini", lambda x: np.nan))

elif lang == "fr":
    scores["KandelMoles"] = float(np.nanmean([kandel_moles_french(t) for t in pred_texts]))

scores


{'BLEU': 3.88,
 'SARI': np.float64(32.5),
 'BERTScore Precision:': 0.8654,
 'BERTScore Recall:': 0.8296,
 'BERTScore F1:': 0.8466,
 'MeaningBERT complex-simple': 0.9977,
 'MeaningBERT complex-prediction': 0.9614,
 'MeaningBERT ximple-prediction': 0.9537,
 'Semantic Similarity complex-simple': np.float32(0.7774),
 'Semantic Similarity complex-pred': np.float32(0.7774),
 'Semantic Similarity reference-pred': np.float32(0.5627),
 'FKGL': 8.404346851985272,
 'FleschReadingEase': 54.73057515574505,
 'SMOG': 10.445266022448656}

## 7) Mostrar resultados y guardar a CSV


In [39]:
import pandas as pd
from datetime import datetime

meta = {
    "dataset": DATASET,
    "model": MODEL_NAME,
    "prompt_type": PROMPT_TYPE,
    "prompt_strategy": PROMPT_STRATEGY,
    "lang": lang,
    "n": len(df),
    "timestamp": datetime.now().isoformat(timespec="seconds"),
}

row = {**meta, **scores}
res_df = pd.DataFrame([row])

print("=== Resultados ===")
display(res_df)

out_name = f"scores_{DATASET}_{MODEL_NAME}_{PROMPT_TYPE}_{PROMPT_STRATEGY}.csv"
res_df.to_csv(out_name, index=False)
print("✅ Guardado:", out_name)


=== Resultados ===


,dataset,model,prompt_type,prompt_strategy,lang,n,timestamp,BLEU,SARI,BERTScore Precision:,...,BERTScore F1:,MeaningBERT complex-simple,MeaningBERT complex-prediction,MeaningBERT ximple-prediction,Semantic Similarity complex-simple,Semantic Similarity complex-pred,Semantic Similarity reference-pred,FKGL,FleschReadingEase,SMOG
0,tsar2024,gemma-3-12b-it,ISO,one,en,117,2026-02-27T09:37:23,3.88,32.5,0.8654,...,0.8466,0.9977,0.9614,0.9537,0.7774,0.7774,0.5627,8.404347,54.730575,10.445266


✅ Guardado: scores_tsar2024_gemma-3-12b-it_ISO_one.csv
